# Coursework Assignment: KNN on Wine DataSet

```
University of London
BSc in Computer Science
CM3015, Machine Learning and Neural Networks
Hudson Leonardo MENDES
hlm12@student.london.ac.uk
```

# 1. Exploratory Data Analysis

In [ ]:
%matplotlib inline
# import modules
from sklearn import datasets import numpy as np
import matplotlib.pyplot as plt import pandas as pd
# load data
wine=datasets.load_wine() #print(wine.DESCR)
# this dataset has 13 features, we will only choose a subset of these
df_wine = pd.DataFrame(wine.data, columns = wine.feature_names )
selected_features = ['alcohol','flavanoids','color_intensity','ash']
# extract the data as numpy arrays of features, X, and target, y
X = df_wine[selected_features].values
y = wine.target

## 1.1. Visualising the Data

In [ ]:
def plot_wine_dataset_scatter(df):
    pass

In [ ]:
plot_wine_dataset_scatter(df=X)

## 1.2. Exploratory Data Analysis under Noise

In [ ]:
r = 42
np.random.seed(r)
XN = X+np.random.normal(0,0.6,X.shape)

In [ ]:
plot_wine_dataset_scatter(df=XN)

## 1.3. (Q1) Exploratory Data Analysis

**Question:** If you were to build a classifier using only two of the available features, which ones would you choose any why?

**Answer:** `<answer>`

## 1.4. (Q2) Data with Noise

**Question:** What do you observe by plotting the data without noise compared to plotting data with added Gaussian noise? 

**Answer:** `<answer>`

#  2. Classifier Implementation

In [ ]:
import numpy as np
from typing import List, Any

class KnnIndex:
    documents: List[Any]
    vectors: np.array

    def __init__(self, documents, targets):
        self.documents = documents
        self.targets = targets
        self.vectors = None #TODO: implement

In [ ]:
def make_knn_classifier(X, y, X_, options):
    """
    Performs a k-NN classification given a set of data.

    :param X: documents representing the training data
    :param y: training target classes/labels
    :param X_: testing documents
    :param y_: predicted outputs
    """
    return KnnIndex(X, y).search(X_)

# 3. Classifier Evaluation

## 3.1. Nested Cross-Validation

## 3.2. Confusion Matrix

# 5. Conclusions

## 5.1. Influence of Noise

## 5.2. Beyond Wine

## 5.3. Final Considerations